In [5]:
## Checking the provided lame dataset
import pandas as pd
import os

givenData = pd.read_table("profile_semantic_trafo_final.txt", delim_whitespace= True)

## Checking for null values
givenData.isnull().any()
# Interpretation: No null values

time      False
start     False
target    False
weight    False
dtype: bool

In [6]:
### Dataset that we will use for the tool

friendNetworks = pd.read_table("Brightkite_edges.txt", delim_whitespace= True, header=None)
friendNetworks.columns = ["Person", "Friend_with"]

check_ins = pd.read_table("Brightkite_totalCheckins.txt", delim_whitespace= True, header=None)
check_ins.columns = ['User', "Check_in_time", "Latitude", "Longitude", "Location_id"]

check_ins["Check_in_time"] = pd.to_datetime(check_ins["Check_in_time"])

In [7]:
## Investigating null values and error values
print("Shapes for friendNetwork and check_ins before preprocessing:", (friendNetworks.shape, check_ins.shape))
def dropNulls(data):
    return data.dropna(axis = 0)

## Latitude and longitudes are not zeros 

def dropZeroLocations(data):
    return data[(data['Latitude'] != 0) & (data['Longitude'] != 0)]

friendNetworks = dropNulls(friendNetworks)
check_ins = dropNulls(dropZeroLocations(check_ins))
print("Shapes for friendNetwork and check_ins after preprocessing:", (friendNetworks.shape, check_ins.shape))

Shapes for friendNetwork and check_ins before preprocessing: ((428156, 2), (4747287, 5))
Shapes for friendNetwork and check_ins after preprocessing: ((428156, 2), (4491074, 5))


In [322]:
#Simple statistics 
times = pd.to_datetime(check_ins["Check_in_time"])
times = pd.DatetimeIndex(check_ins.Check_in_time)
grouped = check_ins.groupby([times.hour])
grouped.describe() #This is just for each hour of the day as a whole

User                                                     \
                  count          mean           std  min      25%      50%   
Check_in_time                                                                
0              239915.0  12856.920905  12949.510597  0.0  2959.00   8288.0   
1              233454.0  12822.841206  12978.330063  0.0  2932.00   8290.0   
2              223643.0  12674.445701  12881.090043  0.0  2900.00   8260.0   
3              214761.0  12709.657070  12732.937227  0.0  2961.00   8557.0   
4              192858.0  12842.705934  12703.443918  0.0  3056.00   8924.0   
5              163894.0  13107.829018  12770.784421  0.0  3180.00   9328.0   
6              140068.0  13481.113038  12847.674258  0.0  3458.00   9797.0   
7              125177.0  14088.334183  13031.307903  0.0  4012.00  10428.0   
8              116013.0  14898.254204  13450.943981  0.0  5472.00  10658.0   
9              109383.0  15196.394970  13446.497717  0.0  6247.00  10823.0   
10             106418.0  15265.818001  13320.921879  0.0  6604.25  10959.0   
11             113017.0  15169.084607  13456.154421  0.0  6277.00  10784.0   
12             130803.0  14845.599826  13459.878516  0.0  5369.00  10613.0   
13             157843.0  14602.720007  13358.277439  0.0  4576.00  10448.0   
14             184374.0  14349.023615  13292.184779  0.0  3882.00  10171.0   
15             199979.0  14079.748294  13430.575448  0.0  3414.00   9574.0   
16             214780.0  13723.525645  13485.534666  0.0  3108.00   8921.0   
17             231002.0  13759.666193  13536.964592  0.0  3085.00   8896.0   
18             238480.0  13665.375155  13507.780459  0.0  3056.00   8698.0   
19             233816.0  13523.075050  13490.299954  0.0  2998.00   8533.0   
20             227744.0  13342.505541  13420.179141  0.0  2976.00   8268.0   
21             225407.0  13332.265271  13368.991333  0.0  2975.00   8308.0   
22             230863.0  13320.667175  13295.884017  0.0  3010.00   8533.0   
23             237382.0  13173.982433  13139.599684  0.0  3006.00   8515.0   

                                  Latitude                ...                 \
                    75%      max     count       mean     ...            75%   
Check_in_time                                             ...                  
0              18128.00  58227.0  239915.0  35.237028     ...      41.208684   
1              17862.00  58205.0  233454.0  34.829271     ...      40.757228   
2              17573.00  58222.0  223643.0  34.481066     ...      40.713175   
3              17275.00  58214.0  214761.0  34.170078     ...      40.437018   
4              17156.00  58186.0  192858.0  33.759914     ...      40.241681   
5              17431.00  58205.0  163894.0  33.490163     ...      40.303310   
6              17760.00  58192.0  140068.0  33.606560     ...      40.760268   
7              18725.00  58192.0  125177.0  34.501009     ...      43.620172   
8              20249.00  58186.0  116013.0  34.958980     ...      48.700000   
9              21323.00  58220.0  109383.0  35.264557     ...      50.795692   
10             21596.00  58220.0  106418.0  35.364946     ...      50.850000   
11             21075.00  58214.0  113017.0  36.161832     ...      50.722518   
12             21458.00  58207.0  130803.0  36.736362     ...      48.791646   
13             21408.00  58226.0  157843.0  36.955519     ...      44.883263   
14             20919.00  58206.0  184374.0  37.287603     ...      43.206460   
15             20069.00  58224.0  199979.0  37.930607     ...      43.093086   
16             19538.25  58198.0  214780.0  38.394718     ...      43.195491   
17             19902.00  58210.0  231002.0  38.582869     ...      43.586332   
18             19709.00  58194.0  238480.0  38.663450     ...      43.593620   
19             19604.00  58215.0  233816.0  38.591830     ...      43.220471   
20             19164.00  58186.0  227744.0  38.223297     ...      42.98

In [323]:
#subsequent commands can be generalized to functions as expected in assignment (read 4B before continuing)
check_ins["Check_in_time"] = pd.to_datetime(check_ins["Check_in_time"])
check_ins_dateindex = check_ins.set_index("Check_in_time")
time_group = check_ins_dateindex.groupby(pd.TimeGrouper('2W')) #Where 2W can be 1S, 1D, 1W, 1M, 2M, 3M, 1Y, etc.. for second, week, month 2 months, 3months, years 

In [324]:
#Code useful for later use
#q.resample('W').groups
#d.mean().to_period().index
time_group.mean() #gives mean of columns over a step of 2 weeks

,User,Latitude,Longitude
Check_in_time,,,
2008-03-23,265.222222,39.778284,-104.965370
2008-04-06,1922.049180,38.166661,-103.702162
2008-04-20,7600.287399,35.978091,-93.977578
2008-05-04,12451.576943,35.085032,-77.593485
2008-05-18,13497.136009,36.395072,-80.152839
2008-06-01,12888.646684,36.791854,-82.459542
2008-06-15,13732.927651,36.449404,-79.815972
2008-06-29,13419.614695,36.124681,-75.383523
2008-07-13,13729.296137,36.169411,-73.762025


In [327]:
time_group.sum() #sum of time steps as explained in the assignment

,User,Latitude,Longitude
Check_in_time,,,
2008-03-23,4774,7.160091e+02,-1.889377e+03
2008-04-06,234490,4.656333e+03,-1.265166e+04
2008-04-20,12244063,5.796070e+04,-1.513979e+05
2008-05-04,219359431,6.180930e+05,-1.366964e+06
2008-05-18,786154184,2.119867e+06,-4.668582e+06
2008-06-01,660272481,1.884810e+06,-4.224320e+06
2008-06-15,776720655,2.061542e+06,-4.514312e+06
2008-06-29,823870405,2.217803e+06,-4.628021e+06
2008-07-13,835153084,2.200185e+06,-4.486944e+06


In [316]:
time_group.groups #gives groupings and groupings can be accessed with the index using time_group.get_group(index) or...

{Timestamp('2008-03-23 00:00:00', freq='2W-SUN'): 18,
 Timestamp('2008-04-06 00:00:00', freq='2W-SUN'): 140,
 Timestamp('2008-04-20 00:00:00', freq='2W-SUN'): 1751,
 Timestamp('2008-05-04 00:00:00', freq='2W-SUN'): 19368,
 Timestamp('2008-05-18 00:00:00', freq='2W-SUN'): 77614,
 Timestamp('2008-06-01 00:00:00', freq='2W-SUN'): 128843,
 Timestamp('2008-06-15 00:00:00', freq='2W-SUN'): 185402,
 Timestamp('2008-06-29 00:00:00', freq='2W-SUN'): 246795,
 Timestamp('2008-07-13 00:00:00', freq='2W-SUN'): 307625,
 Timestamp('2008-07-27 00:00:00', freq='2W-SUN'): 363065,
 Timestamp('2008-08-10 00:00:00', freq='2W-SUN'): 420185,
 Timestamp('2008-08-24 00:00:00', freq='2W-SUN'): 478027,
 Timestamp('2008-09-07 00:00:00', freq='2W-SUN'): 534849,
 Timestamp('2008-09-21 00:00:00', freq='2W-SUN'): 591128,
 Timestamp('2008-10-05 00:00:00', freq='2W-SUN'): 643340,
 Timestamp('2008-10-19 00:00:00', freq='2W-SUN'): 697455,
 Timestamp('2008-11-02 00:00:00', freq='2W-SUN'): 809667,
 Timestamp('2008-11-16 00

In [326]:
list(time_group) #more intuitive view of groupings

,User,Latitude,Longitude
Check_in_time,,,
2008-03-23,4774,7.160091e+02,-1.889377e+03
2008-04-06,234490,4.656333e+03,-1.265166e+04
2008-04-20,12244063,5.796070e+04,-1.513979e+05
2008-05-04,219359431,6.180930e+05,-1.366964e+06
2008-05-18,786154184,2.119867e+06,-4.668582e+06
2008-06-01,660272481,1.884810e+06,-4.224320e+06
2008-06-15,776720655,2.061542e+06,-4.514312e+06
2008-06-29,823870405,2.217803e+06,-4.628021e+06
2008-07-13,835153084,2.200185e+06,-4.486944e+06


In [319]:
#Get users and check ins that are in a certain longitute/latitude threshold
def between_coordinates(data, lat_min, lat_max, long_min, long_max):
    threshold = data[(data["Latitude"].between(lat_min, lat_max)) & data["Longitude"].between(long_min, long_max)]
    return threshold

In [320]:
#Ignore this

#Get friends of friends of a certain user

def get_friends(data, person_number):
    person = data[data["Person"]==person_number]
    for i in list(person["Friend_with"]):
        person = person.append([data[data["Person"]==i]])
    return person.drop_duplicates()
#get_friends(friendNetworks, 0)


In [ ]:
#Kacper's code for getting friends of friends of friends of....
def make_visualisation_great_again(first_person, layer):
    new_person = []
    for i in first_person:
        person = friendNetworks[friendNetworks['Person']==i]
        new_person = new_person + list(person["Friend_with"])
        for i in list(person["Friend_with"]):
            first_dick[layer].append(i)
    new_person = set(new_person)
    first_dick[layer] = set(first_dick[layer])
    
    return new_person
new_person_2 = make_visualisation_great_again([0], 'layer_1')
new_person_3 = make_visualisation_great_again(new_person_2, 'layer_2')


In [ ]:
def make_generalisation_great_again(first_person, number_of_layers):
    count = 0
    while count < (number_of_layers + 1):
        make_visualisation_great_again(first_person, count + 1)
        first_person = new_person_ + str(layer)